<a href="https://colab.research.google.com/github/MaheepChaudhary/GANs_mnist/blob/main/GANs_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
#importing the necessary packages
import tensorflow as tf
import tensorflow.keras as tk
from tensorflow.keras.datasets import mnist 
import imageio
import numpy as np
from sklearn.utils import shuffle
from tensorflow.keras.layers import Dense,Conv2DTranspose,Conv2D,Dropout,Flatten,MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import activations

In [39]:
#preprocessing of dataset(normalization and shuffling of images)
dataset = mnist.load_data(path="mnist.npz")
x_train,y_train,x_test,y_test = dataset[0][0],dataset[0][1],dataset[1][0],dataset[1][1]
x_train = x_train/255.0;x_test = x_test/255.0
x_train,y_train = shuffle(x_train,y_train)
x_test,y_test = shuffle(x_test,y_test)

In [48]:
#generator model
def generator(noise):
  model = Sequential()
  model.add(Dense(noise))
  model.add(tf.keras.layers.Reshape(None,28,28,1))
  model.add(Conv2DTranspose(256,activations = 'relu'))
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(128,activations = 'relu'))
  model.add(Dropout(0.2))
  model.add(Conv2DTranspose(64,activations = 'relu'))
  model.add(Dropout(0.2))
  return model

In [51]:
#discriminator model
def discriminator(image):
  model = Sequential()  
  model.add(Conv2D(64,kernel = (3,3),activation='relu'))
  model.add(Dropout(0.2))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(128,kernel = (3,3),activation='relu'))
  model.add(Dropout(0.3))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Conv2D(256,kernel = (3,3),activation='relu'))
  model.add(Dropout(0.2))
  model.add(MaxPooling2D(pool_size=(2,2)))
  model.add(Flatten())
  model.add(Dense(200,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(100,activation='relu'))
  model.add(Dropout(0.2))
  model.add(Dense(50,activation='relu'))
  model.add(Dense(1,activation='relu'))
  return model

In [58]:
#loss
#loss = tk.losses.binary_crossentropy(from_logits = True)


In [59]:
#optimizers
gen_optimizer = tf.keras.optimizers.Adam(1e-4)
dis_optimizer = tf.keras.optimizers.Adam(1e-4)

In [60]:
#generative_loss
def gen_loss(fake):
  return tk.losses.binary_crossentropy(tf.zeros_like(fake),fake,from_logits = True)

In [61]:
#discriminator_loss
def dis_loss(fake,original):
  real_loss = tk.losses.binary_crossentropy(tf.zeros_like(original),original,from_logits = True)
  fake_loss = tk.losses.binary_crossentropy(tf.zeros_like(fake),fake,from_logits = True)  
  total_loss= real_loss + fake_loss
  return total_loss  

In [ ]:
#optimizer


In [ ]:
#training_Step


In [ ]:
#training_function


In [17]:
#plotting and saving image function


In [56]:
#training occurs
def add(x,y):
  print(x+y)
x = [1,2,3,4,4]
y = [4,1,1,1,1]
add(x,y)

[1, 2, 3, 4, 4, 4, 1, 1, 1, 1]
